In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

os.chdir('../')

In [22]:
import torch
torch.tensor([1, 2, 3]).view(-1, 1) .shape

torch.Size([3, 1])

In [3]:
from src.data_loader import load_hp_params
from src.plotters import Parallel_Coordinates

import matplotlib.pyplot as plt

/Users/gbemidebe/miniconda3/envs/knnMoleSearch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
!pwd

/Users/gbemidebe/Documents/GitHub/knnGuidedDesign


In [18]:
from tbparse import SummaryReader
log_dir = "reports/lightning_logs/knnMoleculeVAE/version_1/"
reader = SummaryReader(log_dir, pivot=True)
df = reader.scalars
print(df)

     step                  epoch  hp_metric  train_loss_epoch  \
0       0                    NaN       -1.0               NaN   
1       1                    NaN        NaN               NaN   
2       2                    NaN        NaN               NaN   
3       3                    NaN        NaN               NaN   
4       4                    NaN        NaN               NaN   
..    ...                    ...        ...               ...   
847  2367         [295.0, 295.0]        NaN       1023.825195   
848  2375         [296.0, 296.0]        NaN       1033.995483   
849  2383         [297.0, 297.0]        NaN       1024.336548   
850  2391         [298.0, 298.0]        NaN       1014.230957   
851  2399  [299.0, 299.0, 299.0]        NaN       1010.192566   

     train_loss_step  val_loss_epoch  val_loss_step  
0                NaN             NaN   10809.890625  
1                NaN             NaN    9811.266602  
2                NaN             NaN   10819.225586  
3  

In [16]:
hpp_df = load_hp_params("reports/lightning_logs/knnMoleculeVAE/version_*")

run_0 has 1291 epochs


ConstructorError: could not determine a constructor for the tag 'tag:yaml.org,2002:python/object:pytorch_lightning.accelerators.mps.MPSAccelerator'
  in "reports/lightning_logs/knnMoleculeVAE/version_0/hparams.yaml", line 4, column 9

In [ ]:
fig,ax = plt.subplots(figsize=(12,5))
pcd = Parallel_Coordinates(dataframe=hpp_df, best_itr=45,ax=ax).plot()
plt.tight_layout()
# plt.savefig("images/hyperparams.png", dpi=200, facecolor='w', edgecolor='none',bbox_inches='tight')
# plt.show()